In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import codecs, json
from tqdm import tqdm
import numpy as np
from datetime import datetime
import time
import math
from sklearn.decomposition import FastICA

import sys
sys.path.append('drive/MyDrive/BmiResearch')
from constants import constants

In [ ]:
constants.SIGNAL_PROCESSING

['SLF', 'ASR', 'NOSP']

In [ ]:
for sp in constants.SIGNAL_PROCESSING:
  output_path = f'{constants.PREPROCESSED_DATASET_PATH}/{sp}_ICA'
  print('output_path: ', output_path)

  if sp == 'NOSP':
    actual_base_dataset_path = constants.BASE_DATASET_PATH
  else:
    actual_base_dataset_path = f'{constants.PREPROCESSED_DATASET_PATH}/{sp}'
  print("actual_base_dataset_path = ", actual_base_dataset_path)

  for subject in sorted(os.listdir(actual_base_dataset_path)):
    print(subject)
    experiment_settings = dict()
    experiment_settings['general_params'] = {'low_filter':constants.low_filter,
                                            'high_filter':constants.high_filter,
                                            'frequency':constants.freq,
                                            'minutes_for_test':constants.minutes_for_test,
                                            'window_size':constants.window_size,
                                            'overlap':constants.overlap,
                                            'EEG_CHANNELS':constants.EEG_CHANNELS}
    experiment_settings['feature_extraction'] = 'ICA'
    experiment_settings['DateTime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    experiment_settings['BASE_DATASET_PATH'] = constants.BASE_DATASET_PATH
    experiment_settings['OUTPUT_PATH'] = output_path

    output_path_subject = (f'{output_path}/{subject}')
    os.makedirs(output_path_subject)

    experiment_settings[subject] = dict()

    # Fit ICA
    fit_df = codecs.open(f'{actual_base_dataset_path}/{subject}/X_fit.json', 'r', encoding='utf-8').read()
    fit_df = json.loads(fit_df)
    fit_df = np.array(fit_df)

    # !!!!! ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
    one_ica = FastICA(random_state=11, max_iter=2000, tol=0.0001)
    print("fit_df.shape", fit_df.shape)
    one_ica.fit(fit_df)

    ica_fit = one_ica.transform(fit_df)

    ica_fit = ica_fit.tolist()
    json.dump(ica_fit, codecs.open(f'{output_path_subject}/X_fit.json', 'w', encoding='utf-8'),
      separators=(',', ':'),
      sort_keys=True,
      indent=4)

    # process train
    chanks_train = codecs.open(f'{actual_base_dataset_path}/{subject}/X_train_chunks.json', 'r', encoding='utf-8').read()
    chanks_train = json.loads(chanks_train)
    chanks_train = np.array(chanks_train)

    ica_train = []
    for i in range(chanks_train.shape[0]):
        ica_train.append(one_ica.transform(chanks_train[i].T).T)
    ica_train = np.array(ica_train)

    ica_train = ica_train.tolist()
    json.dump(ica_train, codecs.open(f'{output_path_subject}/X_train_chunks.json', 'w', encoding='utf-8'),
      separators=(',', ':'),
      sort_keys=True,
      indent=4)

    # process test
    chanks_test = codecs.open(f'{actual_base_dataset_path}/{subject}/X_test_chunks.json', 'r', encoding='utf-8').read()
    chanks_test = json.loads(chanks_test)
    chanks_test = np.array(chanks_test)

    starttime = time.perf_counter()
    ica_test = []
    for i in range(chanks_test.shape[0]):
        ica_test.append(one_ica.transform(chanks_test[i].T).T)

    proc_end = time.perf_counter() - starttime
    ica_test = np.array(ica_test)

    proc_1ch_s = round(proc_end / chanks_test.shape[0], 5)
    experiment_settings[subject]['proc_1_test_ch_s'] = proc_1ch_s
    experiment_settings[subject]['proc_test_s'] = round(proc_end, 5)
    experiment_settings[subject]['len_test'] = chanks_test.shape[0]
    experiment_settings[subject]['output_path_subject'] = output_path_subject

    ica_test = ica_test.tolist()
    json.dump(ica_test, codecs.open(f'{output_path_subject}/X_test_chunks.json', 'w', encoding='utf-8'),
      separators=(',', ':'),
      sort_keys=True,
      indent=4)

    json.dump(experiment_settings, codecs.open(f'{output_path_subject}/experiment_settings.json', 'w', encoding='utf-8'),
      separators=(',', ':'),
      sort_keys=True,
      indent=4)